In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/1000shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.5, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


# Loss

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [13]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [14]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

In [16]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : teacher_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.0529 | MainLoss:0.9226 | SPLoss:1.1501 | CLSLoss:1.5266 | top1:51.3867 | AUROC:0.0000
Test | 32/16 | Loss:0.8152 | MainLoss:0.6924 | SPLoss:1.0818 | CLSLoss:1.4672 | top1:53.5897 | AUROC:0.5697
Test | 129/16 | Loss:0.7885 | MainLoss:0.6657 | SPLoss:1.0818 | CLSLoss:1.4672 | top1:92.0156 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.130000
Train | 16/16 | Loss:0.7988 | MainLoss:0.6939 | SPLoss:0.9045 | CLSLoss:1.4402 | top1:50.9067 | AUROC:0.0000
Test | 32/16 | Loss:0.7782 | MainLoss:0.6914 | SPLoss:0.7268 | CLSLoss:1.4083 | top1:55.3462 | AUROC:0.5661
Test | 129/16 | Loss:0.7041 | MainLoss:0.6173 | SPLoss:0.7268 | CLSLoss:1.4083 | top1:99.0872 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.160000
Train | 16/16 | Loss:0.7637 | MainLoss:0.6916 | SPLoss:0.5830 | CLSLoss:1.3774 | top1:52.0800 | AUROC:0.0000
Test | 32/16 | Loss:0.7462 | MainLoss:0.6887 | SPLoss:0.4408 | CLSLoss:1.3435 | top1:54.8590 | AUROC:0.5690
Test | 129/16 | Loss:0.5312 | MainL

Test | 129/16 | Loss:51.0522 | MainLoss:0.8590 | SPLoss:501.8930 | CLSLoss:0.3894 | top1:51.5016 | AUROC:0.5263

Epoch: [24 | 1000] LR: 0.399858
Train | 16/16 | Loss:31.7946 | MainLoss:0.3068 | SPLoss:314.8351 | CLSLoss:0.4281 | top1:88.1867 | AUROC:0.0000
Test | 32/16 | Loss:18.0294 | MainLoss:0.4889 | SPLoss:175.3637 | CLSLoss:0.4162 | top1:77.6410 | AUROC:0.9645
Test | 129/16 | Loss:18.4293 | MainLoss:0.8887 | SPLoss:175.3639 | CLSLoss:0.4162 | top1:54.0467 | AUROC:0.5600

Epoch: [25 | 1000] LR: 0.399833
Train | 16/16 | Loss:14.7331 | MainLoss:0.3734 | SPLoss:143.5577 | CLSLoss:0.3910 | top1:84.0800 | AUROC:0.0000
Test | 32/16 | Loss:8.6165 | MainLoss:0.3180 | SPLoss:82.9413 | CLSLoss:0.4367 | top1:87.0000 | AUROC:0.9691
Test | 129/16 | Loss:9.2550 | MainLoss:0.9565 | SPLoss:82.9413 | CLSLoss:0.4367 | top1:56.0343 | AUROC:0.5964

Epoch: [26 | 1000] LR: 0.399807
Train | 16/16 | Loss:7.8735 | MainLoss:0.3401 | SPLoss:75.2932 | CLSLoss:0.4051 | top1:86.1067 | AUROC:0.0000
Test | 32/16 

Train | 16/16 | Loss:9.6322 | MainLoss:0.3692 | SPLoss:92.5881 | CLSLoss:0.4257 | top1:84.2933 | AUROC:0.0000
Test | 32/16 | Loss:11.3284 | MainLoss:0.3019 | SPLoss:110.2292 | CLSLoss:0.3582 | top1:89.3718 | AUROC:0.9578
Test | 129/16 | Loss:11.8985 | MainLoss:0.8720 | SPLoss:110.2291 | CLSLoss:0.3582 | top1:51.8536 | AUROC:0.6187

Epoch: [47 | 1000] LR: 0.398792
Train | 16/16 | Loss:9.9858 | MainLoss:0.3772 | SPLoss:96.0442 | CLSLoss:0.4132 | top1:84.2933 | AUROC:0.0000
Test | 32/16 | Loss:5.4715 | MainLoss:0.3807 | SPLoss:50.8730 | CLSLoss:0.3526 | top1:87.0000 | AUROC:0.9476
Test | 129/16 | Loss:6.1096 | MainLoss:1.0188 | SPLoss:50.8730 | CLSLoss:0.3526 | top1:48.9439 | AUROC:0.4818

Epoch: [48 | 1000] LR: 0.398722
Train | 16/16 | Loss:3.4841 | MainLoss:0.3032 | SPLoss:31.7641 | CLSLoss:0.4483 | top1:88.5067 | AUROC:0.0000
Test | 32/16 | Loss:2.0690 | MainLoss:0.2714 | SPLoss:17.9306 | CLSLoss:0.4574 | top1:89.5513 | AUROC:0.9629
Test | 129/16 | Loss:2.8579 | MainLoss:1.0602 | SPLos

Test | 129/16 | Loss:4.3726 | MainLoss:1.3552 | SPLoss:30.1153 | CLSLoss:0.5858 | top1:56.3956 | AUROC:0.7199

Epoch: [69 | 1000] LR: 0.039680
Train | 16/16 | Loss:3.0073 | MainLoss:0.1422 | SPLoss:28.5919 | CLSLoss:0.5900 | top1:94.8533 | AUROC:0.0000
Test | 32/16 | Loss:2.8614 | MainLoss:0.1633 | SPLoss:26.9218 | CLSLoss:0.5911 | top1:93.3846 | AUROC:0.9841
Test | 129/16 | Loss:4.3260 | MainLoss:1.6279 | SPLoss:26.9218 | CLSLoss:0.5911 | top1:54.0779 | AUROC:0.7228

Epoch: [70 | 1000] LR: 0.039669
Train | 16/16 | Loss:2.6926 | MainLoss:0.1298 | SPLoss:25.5677 | CLSLoss:0.5940 | top1:95.3067 | AUROC:0.0000
Test | 32/16 | Loss:2.5869 | MainLoss:0.1710 | SPLoss:24.0986 | CLSLoss:0.5983 | top1:93.0769 | AUROC:0.9837
Test | 129/16 | Loss:4.0505 | MainLoss:1.6347 | SPLoss:24.0986 | CLSLoss:0.5983 | top1:54.7414 | AUROC:0.7295

Epoch: [71 | 1000] LR: 0.039657
Train | 16/16 | Loss:2.4164 | MainLoss:0.1200 | SPLoss:22.9045 | CLSLoss:0.6022 | top1:95.5200 | AUROC:0.0000
Test | 32/16 | Loss:2.3

Test | 129/16 | Loss:8.1745 | MainLoss:1.6796 | SPLoss:64.8840 | CLSLoss:0.6496 | top1:54.3769 | AUROC:0.6421

Epoch: [92 | 1000] LR: 0.039372
Train | 16/16 | Loss:6.2970 | MainLoss:0.1382 | SPLoss:61.5238 | CLSLoss:0.6472 | top1:94.8533 | AUROC:0.0000
Test | 32/16 | Loss:5.9806 | MainLoss:0.1896 | SPLoss:57.8452 | CLSLoss:0.6426 | top1:92.6410 | AUROC:0.9831
Test | 129/16 | Loss:7.3840 | MainLoss:1.5930 | SPLoss:57.8451 | CLSLoss:0.6426 | top1:55.4050 | AUROC:0.7428

Epoch: [93 | 1000] LR: 0.039356
Train | 16/16 | Loss:5.6166 | MainLoss:0.1162 | SPLoss:54.9391 | CLSLoss:0.6449 | top1:95.5733 | AUROC:0.0000
Test | 32/16 | Loss:5.3761 | MainLoss:0.1782 | SPLoss:51.9143 | CLSLoss:0.6476 | top1:93.3462 | AUROC:0.9841
Test | 129/16 | Loss:6.9717 | MainLoss:1.7738 | SPLoss:51.9144 | CLSLoss:0.6476 | top1:54.8536 | AUROC:0.7473

Epoch: [94 | 1000] LR: 0.039340
Train | 16/16 | Loss:5.0310 | MainLoss:0.1023 | SPLoss:49.2226 | CLSLoss:0.6492 | top1:96.2933 | AUROC:0.0000
Test | 32/16 | Loss:4.8